# DSA - inner collisions

Find two (meaningful*) messages m_1 and m_2, for which DSA public parameters (p, q, g) exist such that both messages have the same signature (r, s) regardless of the private key value. The signature of m_1 is thus at the same time a valid signature of m_2 and vice versa. Find the parameters (p, q, g) and show the collision of both signatures for a randomly chosen private key.

In [1]:
from random import randrange
from functools import lru_cache
import hashlib
from sympy import isprime

In [2]:
N = 160
L = 1024

TRY = 1000

In [3]:
def random_word(words):
    return words[randrange(len(words))]

@lru_cache(maxsize=None)
def H(msg: str) -> int:
    binary_digest = hashlib.sha1(msg.encode()).digest()
    return int.from_bytes(binary_digest, byteorder='big')

def sign(msg, p, q, g, x):
    k = randrange(1, q)
    r = pow(g, k, p) % q
    s = pow(k, -1, q) * (H(msg) + x * r) % q
    return r, s

def verify(msg, r, s, p, q, g, y):
    if not (0 < r < q and 0 < s < q):
        return False
    w = pow(s, -1, q)
    u1 = H(msg) * w % q
    u2 = r * w % q
    v = pow(g, u1, p) * pow(y, u2, p) % p % q
    return v == r

In [4]:
with open("words.txt", "r") as f:
    words = f.read().splitlines()
    found = False
    while not found:
        m1 = random_word(words)
        h1 = H(m1)
        for _ in range(TRY):
            m2 = random_word(words)
            if m1 == m2:
                continue
            h2 = H(m2)
            q = abs(h1 - h2)
            if q == 0:
                print(f"Hash collision: {m1}, {m2}")
            if q.bit_length() == N and isprime(q):
                for _ in range(TRY):
                    p = q * randrange(2**(L-1) // q, 2**L // q) + 1
                    if p.bit_length() == L and isprime(p):
                        for _ in range(TRY):
                            h = randrange(2, p)
                            g = pow(h, (p-1) // q, p)
                            if g != 1:
                                found = True
                                break
                        if found:
                            break
                if found:
                    break

print(f"Params (\n  {p=},\n  {q=},\n  {g=}\n)")
print(f"Message 1: {m1}")
print(f"Message 2: {m2}")

Params (
  p=144083100281591605451159397187506583965112822883675521484931524139635771984608708499464096642385083104026267768955620967886854685633784884573829854085039348920739715611615637183606297329076755354656158865435150044350349455098540489496947701329454566423498565650640731334757484420935076482274031393490044409201,
  q=914674423860552342366706961206651745430086824819,
  g=139052240548920405033528506104922290650045096943578232446806025389843541915895195942892272930869974460755457769790572026219638852338144649806033928233025707245558897462001332592324664260910708069112862982206299344766227970762903416738950338956068049303269811550582181231165709171266272839206531940302998255920
)
Message 1: anutraminosa
Message 2: panelwork


In [5]:
x = randrange(1, q - 1)
y = pow(g, x, p)
print(f"Private key: {x}")
print(f"Public key: {y}")

Private key: 16620377147184025480670071234175520857345814898
Public key: 140625364599248349944030445607223516442442924997760252892684821306636315280781187364935892310222329130438040465854554910207252615162677241469930617379977579994914112589503345443934769337645737982562812916986600157862095911329285824883377138369551040983190823152176340047907436791745392148322391975461691197283


In [6]:
r1, s1 = sign(m1, p, q, g, x)
r2, s2 = sign(m2, p, q, g, x)
print(f"Signature 1: (r={r1}, s={s1})")
print(f"Signature 2: (r={r2}, s={s2})")
print(f"Message 1 signature 2: {verify(m1, r2, s2, p, q, g, y)}")
print(f"Message 2 signature 1: {verify(m2, r1, s1, p, q, g, y)}")

Signature 1: (r=151981894968631690348237310436893841541784460652, s=170529898577311174642194170725070483465027253671)
Signature 2: (r=203772632167043230177126302809178766010218603070, s=404002800828249734571294531062430619481366360344)
Message 1 signature 2: True
Message 2 signature 1: True
